In [1]:
!pip install psycopg2-binary
!pip install pyarrow
!pip install pyathena fastparquet s3fs

In [2]:
import pandas as pd
import math
from tqdm import tqdm
from pyathena import connect  
from pyathena.pandas.util import as_pandas
from pyathena.pandas.cursor import PandasCursor
from IPython.display import display, clear_output

# PEGA A BASE DE TPV DOS ULTIMOS 160 DIAS

In [3]:
data_calculo = '2021-03-08'
id_oferta = '49'
tipo_oferta = 'ms'
end_date = pd.to_datetime(data_calculo)
start_date = end_date + pd.Timedelta(-159, unit='D')

In [4]:
# data_calculo = datetime.strptime(str(data_calculo_integer), '%Y%m%d').strftime('%Y-%m-%d')
def athena_to_df(query):

    cursor = connect(s3_staging_dir='s3://stone-sandbox-datalake/Heads/athena-query-results/',
                    region_name='us-east-1',
                    work_group = 'Heads').cursor(PandasCursor)
            
    df = cursor.execute(query).as_pandas()
    print(f'Query ID: {cursor._query_id}')
    cursor.close()
    return df

In [5]:
query = f'''
    select
        t1.cnpj_or_cpf,
        date_parse(cast(t3.transactiondate as varchar), '%Y%m%d') as transaction_date,
        SUM(t3.tpv - t3.mdr) as tpv_net_mdr
    from heads.leads_pre_revisao as t1
    left join analytics.stonedw_dimaffiliation as t2 on t1.cnpj_or_cpf = t2.clientcnpjorcpf
    left join analytics.stonedw_facttpv as t3 on t2.affiliationkey = t3.affiliationkey
    where id_oferta = {id_oferta}
    and date_parse(cast(t3.transactiondate as varchar), '%Y%m%d') <= date('{data_calculo}')
    and date_parse(cast(t3.transactiondate as varchar), '%Y%m%d') > date('{data_calculo}') - interval '160' day
    group by 1,2
    order by 1,2
'''

In [6]:
bd_tpv = athena_to_df(query)

Query ID: c72393a0-cd66-4773-8e7a-67690cf7f726


In [7]:
bd_tpv

,cnpj_or_cpf,transaction_date,tpv_net_mdr
0,1009831000197,2020-09-30,1137.9151
1,1009831000197,2020-10-01,1792.0271
2,1009831000197,2020-10-02,957.6189
3,1009831000197,2020-10-03,1344.9659
4,1009831000197,2020-10-05,620.6250
...,...,...,...
181059,9956191663,2021-03-04,2292.9212
181060,9956191663,2021-03-05,2016.5930
181061,9956191663,2021-03-06,3155.5017
181062,9956191663,2021-03-07,2372.7246


In [8]:
def reindex_to_end_date(df):
    dates = pd.date_range(df.index.min(), end_date)
    return df.reindex(dates)

def reindex_to_start_date(df):
    dates = pd.date_range(start_date, end_date)
    return df.reindex(dates)


bd_tpv.set_index('transaction_date', inplace=True)
bd_tpv = bd_tpv.groupby('cnpj_or_cpf').apply(reindex_to_end_date)

bd_tpv.reset_index(0, drop=True, inplace=True)
bd_tpv['cnpj_or_cpf'].fillna(method='ffill', inplace=True)
bd_tpv['tpv_net_mdr'].fillna(0, inplace=True)

bd_tpv = bd_tpv.groupby('cnpj_or_cpf').apply(reindex_to_start_date)
bd_tpv.reset_index(0, drop=True, inplace=True)
bd_tpv['cnpj_or_cpf'].fillna(method='bfill', inplace=True)

bd_tpv.index.name = 'transaction_date'
bd_tpv.reset_index(inplace=True)

bd_tpv = pd.pivot_table(bd_tpv, index='cnpj_or_cpf', values='tpv_net_mdr', columns='transaction_date')

bd_tpv.sort_index(axis=1, ascending=False, inplace=True)
column_names = ['t_' + str(d) for d in range(1, 161)]
bd_tpv.columns = column_names

bd_tpv.reset_index(inplace= True)

In [9]:
bd_tpv

,cnpj_or_cpf,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_151,t_152,t_153,t_154,t_155,t_156,t_157,t_158,t_159,t_160
0,1009831000197,3214.8944,3340.4758,6195.9233,3410.7164,5363.0435,3081.0654,3129.5138,4006.2658,3716.2412,...,1150.4554,1510.1769,1035.0514,633.5494,620.6250,0.0000,1344.9659,957.6189,1792.0271,1137.9151
1,10136246451,432.7840,0.0000,0.0000,507.5720,313.2420,58.5660,177.2180,0.0000,0.0000,...,137.8540,298.1855,107.3710,92.7295,166.9570,0.0000,166.6370,316.3242,285.3690,343.0350
2,10142871000101,2449.1719,0.0000,9760.1180,3447.0500,882.0780,6060.0422,3434.8920,4497.3897,0.0000,...,5095.4240,2943.4868,4465.8588,1989.9748,6574.2440,0.0000,1484.4418,3479.2375,2548.2375,2542.0500
3,10169264000127,0.0000,0.0000,0.0000,1228.8000,536.8000,3058.6185,981.8000,2431.2500,0.0000,...,3977.8637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10217635000107,0.0000,4049.7723,2851.1165,1286.3854,1343.6632,1312.6487,458.9071,0.0000,3564.8925,...,2119.7210,1702.9928,1625.8662,1103.7055,0.0000,3623.3451,3583.9261,2078.8216,1321.9974,2709.4452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,97544652000129,19.5736,0.0000,184.8890,54.4914,154.0586,46.6115,0.0000,111.8485,0.0000,...,75.9018,88.0830,323.8950,54.9832,23.3352,0.0000,583.7916,166.1620,50.7945,101.3108
1464,97551608000146,0.0000,0.0000,0.0000,954.6370,227.0100,21.6153,854.7687,345.3513,0.0000,...,626.7450,925.0746,748.1460,54.2850,731.1668,0.0000,0.0000,1246.6117,985.1870,752.4000
1465,98155814653,285.6774,664.9804,578.3314,574.3989,623.2680,561.1636,261.7888,532.8048,601.8320,...,474.5466,353.8483,286.8455,350.6739,298.5388,552.2048,298.4386,336.5625,236.9883,301.4589
1466,9913313000105,1331.5631,1170.7093,1698.5095,1782.4047,1330.1231,1097.5617,1281.7601,1582.5784,761.6131,...,1131.9641,1510.8220,1593.9587,1018.3747,524.4864,1091.5954,936.2493,1070.8912,1069.1808,928.1218


# Faz as simulacaos para cada um dos vetores de TPV

In [10]:
C = dict()
C['taxa_iof'] = 0.000041

In [11]:
# filename = f'./inputs/{simulacao_input_filename}.csv'
# Oferta = dict((serie['cnpj_or_cpf'], serie) for serie in pd.read_csv(filename, encoding = "ISO-8859-1", dtype={'cnpj_or_cpf':str}).to_dict('record'))

In [12]:
query = f'''
    select 
        cnpj_or_cpf,
        rating,
        tpv_m0_ajustado,
        tpv_ref,
        limite,
        taxa_juros,
        prazo,
        retencao
    from heads.leads_pre_revisao
    where id_oferta = {id_oferta}
'''

In [13]:
# ## BACKTEST ########

# query = f'''
#     select 
#         cnpj_or_cpf,
#         rating,
#         tpv_m0_ajustado,
#         tpv_ref,
#         limite,
#         taxa_juros,
#         prazo,
#         retencao
#     from heads.leads_pre_revisao as t1
#     inner join (
#         select distinct
#             cnpj
#         from kpis.credit_kpis 
#         where data_calculo = (select max(data_calculo) from kpis.credit_kpis)
#         ) as t2 on t1.cnpj_or_cpf = t2.cnpj
#     where id_oferta = {id_oferta}
# '''

In [14]:
bd_oferta = athena_to_df(query)

Query ID: 91d66d44-b909-4623-8cb3-574058bbdf91


In [15]:
Oferta = dict((serie['cnpj_or_cpf'], serie) for serie in bd_oferta.to_dict('record'))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [16]:
Vetores_TPV = dict((dt[0],[i for i in dt[1:] if i == i]) for dt in bd_tpv.to_dict('split')['data'])

In [17]:
class Movimentos:
    def __init__(self, limite, taxa_juros, retencao, projecao_tpv_lim):

                self.flag ='Performing'
                self.saldo = dict((key,[val]) for key,val in zip(['total','principal','imposto','juros'],[limite,limite,0,0]))
                self.flow = dict((key,[]) for key in self.saldo.keys())
                self.adicao = dict((key,[]) for key in self.saldo.keys())                           

                #Projecao da coleta baseado na TPV e retencao contratual
                services = 0.02 # Premissa taxa RAV
                coleta_bruta_projecao = [(1 - services) * retencao * item for item in projecao_tpv_lim]
                 
                #Ajuste aos juros diarios
                juros_diario = (1 + taxa_juros) ** (1 / 30) - 1
                
                #Movimentdo dos valores basedos nas projecoes
                for gr_item in coleta_bruta_projecao:
                    
                        #Calculo dos juros
                        self.adicao['juros'] += [self.saldo['total'][-1] * juros_diario]
                        #Calculo do IOF
                        self.adicao['imposto'] += [self.saldo['total'][-1] * C['taxa_iof']]
                        #Inicialisacao Principal
                        self.adicao['principal'] += [0] #Isto e mantido em caso de no futuro principal ser incrementado
                        #Computando Total
                        self.adicao['total'] += [sum([self.adicao[key][-1] for key in ['imposto','juros','principal'] ])]
                        
                        #Modificacao da coleta se sigma e diferente de zero
                        item = gr_item * 1
                        
                        #Alocacao das coletas
                        for key in ['imposto','juros','principal']: #Prioridade de pagamentos
                            self.flow[key] += [min( item, self.adicao[key][-1] + self.saldo[key][-1] )]
                            item -= self.flow[key][-1]
                        self.flow['total'] += [sum([self.flow[key][-1] for key in ['imposto','juros','principal'] ])]
    
                        #Movimento do saldo
                        for key in self.saldo.keys():
                            self.saldo[key] += [self.saldo[key][-1] + self.adicao[key][-1] - self.flow[key][-1]]
                            
                        #Termino quando sado e zero                    
                        if round(self.saldo['total'][-1],3) == 0:                    
                            break
                
                if sum([1 for item in self.saldo['total'] if item > self.saldo['total'][0]]) > 59: 
                    self.flag ='Default'
                elif self.saldo['total'][-1] > limite:
                    self.flag ='Dificuldade'
                    
                self.arreas = sum([1 for item in self.saldo['juros'] if item > 0])

In [18]:
def roda_simulacao(limite, taxa_juros, retencao, vetor_tpv):
    qtd_cenarios = 0
    tot_default = 0
    tot_dificuldade = 0
    tot_arreas = 0
    tot_receita = 0
    
    for idx in range(0,len(vetor_tpv)-59):
        
        tpv = [i for i in vetor_tpv[idx:idx+60]]
        obj_mov = Movimentos(limite, taxa_juros, retencao, tpv)
        qtd_cenarios += 1
               
        if obj_mov.flag == 'Default':
            tot_default += 1
            tot_receita -= obj_mov.saldo['total'][0] * 0.5
        elif obj_mov.flag == 'Dificuldade':
            tot_dificuldade += 1
            tot_receita -= obj_mov.saldo['total'][0] * 0.5
        elif obj_mov.flag == 'Performing':
            tot_receita += sum(obj_mov.flow['juros'])
            
        tot_arreas += obj_mov.arreas

    prob_default = round(tot_default / qtd_cenarios, 2)
    prob_dificuldade = round(tot_dificuldade / qtd_cenarios, 2)
    avg_arreas = round(tot_arreas / qtd_cenarios, 2)
    avg_receita = round(tot_receita / qtd_cenarios, 2)

    return prob_default, prob_dificuldade, avg_arreas, avg_receita, qtd_cenarios
    

In [19]:
## Run default probabilities for given leads

for cnpj_or_cpf, oferta in tqdm(Oferta.items()):
    prob_default = 0
    prob_dificuldade = 0
    avg_arreas = 0
    avg_receita = 0
    status = 'ok'
    acao = 'ok'
    qtd_cenarios = 0
    
    limite = oferta['limite']
    taxa_juros = oferta['taxa_juros']
    retencao = oferta['retencao']
    tpv_m0_ajustado = oferta['tpv_m0_ajustado']
    tpv_ref = oferta['tpv_ref']
    
    novo_limite = limite
    nova_retencao = retencao
    novo_tpv_ref = tpv_ref
    
    # limite minimo
    limite_min = max(1000, limite / 3)
    
    ## PASSO 1 - CALCULA O TPV MINIMO DA OFERTA ATUAL E A TOLERÂNCIA VS O TPV M0
    tpv_min = limite * taxa_juros / retencao
    fora_tolerancia = (tpv_m0_ajustado < tpv_min) * 1
    
    if (fora_tolerancia == 1):
        status = 'fora_tolerancia'
        novo_tpv_ref = tpv_m0_ajustado
        ratio_limite_tpv = limite / tpv_ref
        novo_limite = ratio_limite_tpv * novo_tpv_ref
        # por definição a retenção se mantém a mesma, dado que alteramos o limite e o TPV na mesma proporção
    
    if cnpj_or_cpf not in Vetores_TPV: #Algumas ofertas nao estao no Vetores_TPV
        status = 'vetor_nao_encontrado'
    else:
        vetor_tpv = Vetores_TPV[cnpj_or_cpf]
        if len(vetor_tpv) <= 60:
            status = 'vetor_pequeno'
            acao = 'nao_emprestar'
        else:
            ## PASSO 2 - SIMULA AS CONDIÇÕES ATUAIS DE CADA OFERTA
            prob_default, prob_dificuldade, avg_arreas, avg_receita, qtd_cenarios = roda_simulacao(novo_limite, taxa_juros, nova_retencao, vetor_tpv)

            ## PASSO 3 - AJUSTA A RETENCÃO ATÉ O MÁXIMO DE 20%          
            while ((round(math.floor(nova_retencao * 100) / 100 + 0.01, 2) <= 0.1) & ((prob_default + prob_dificuldade) >= 0.1)):

                nova_retencao = round(math.floor(nova_retencao * 100) / 100 + 0.01, 2)
                prob_default, prob_dificuldade, avg_arreas, avg_receita, qtd_cenarios = roda_simulacao(novo_limite, taxa_juros, nova_retencao, vetor_tpv)

            ## PASSO 4 - REDUZ O LIMITE CASO A RETENCAO NÃO TENHA SIDO SUFICIENTE
            limite_step = max(1000, round((limite / 20) / 1000) * 1000) # Vou quebrar em 20 steps para não demorar muito em limites muito altos
            while ((math.floor(novo_limite / 1000) * 1000 - limite_step >= limite_min) & ((prob_default + prob_dificuldade) >= 0.1)):

                novo_limite = max(0, math.floor(novo_limite / 1000) * 1000 - limite_step)
                prob_default, prob_dificuldade, avg_arreas, avg_receita, qtd_cenarios = roda_simulacao(novo_limite, taxa_juros, nova_retencao, vetor_tpv)

            if ((nova_retencao > retencao) & (novo_limite < limite)):
                status = 'ajuste_retencao_ajuste_limite'
            elif (novo_limite < limite):
                status = 'ajuste_limite'
            elif (nova_retencao > retencao):
                status = 'ajuste_retencao'

            if (((prob_default + prob_dificuldade) >= 0.1) or (novo_limite < limite_min)):
                acao = 'nao_emprestar'

            ## PASSO 5 - AJUSTA O TPV REF DADA A RETENCAO TARGET
            novo_tpv_ref = tpv_ref * (retencao / nova_retencao) * (novo_limite / limite)

    ## AJUSTAR O TPV REF DADA A MUDANCA DE RETENCAO E LIMITE        
    
    Oferta[cnpj_or_cpf]['prob_default'] = prob_default
    Oferta[cnpj_or_cpf]['prob_dificuldade'] = prob_dificuldade
    Oferta[cnpj_or_cpf]['avg_arreas'] = avg_arreas
    Oferta[cnpj_or_cpf]['avg_receita'] = avg_receita
    Oferta[cnpj_or_cpf]['qtd_cenarios'] = qtd_cenarios
    Oferta[cnpj_or_cpf]['status'] = status
    Oferta[cnpj_or_cpf]['acao'] = acao
    Oferta[cnpj_or_cpf]['nova_retencao'] = nova_retencao
    Oferta[cnpj_or_cpf]['novo_limite'] = novo_limite
    Oferta[cnpj_or_cpf]['novo_tpv_ref'] = novo_tpv_ref

100%|██████████| 1468/1468 [01:21<00:00, 17.92it/s]


# Gera outputs pronto para serem enviados

In [20]:
df_output = pd.DataFrame(list(Oferta.values()))

In [21]:
df_output['rating'] = df_output['rating'].astype(str)

In [22]:
df_output.groupby('status')['cnpj_or_cpf'].count()

status
ajuste_limite                      15
ajuste_retencao                   101
ajuste_retencao_ajuste_limite      45
ok                               1307
Name: cnpj_or_cpf, dtype: int64

In [23]:
output_file_data = data_calculo.replace('-', '')
df_output.to_csv(f'./outputs/revisao_lead_{tipo_oferta}_{id_oferta}_{output_file_data}.csv', index=False)

In [35]:
Vetores_TPV['6967643000195']

[0.0,
 731.775,
 418.39,
 1267.975,
 3242.255,
 2226.077500000001,
 1255.1971,
 0.0,
 532.4,
 804.37,
 1741.06,
 2937.235,
 798.6175,
 2534.655,
 395.4,
 0.0,
 0.0,
 153.2175,
 1169.2,
 632.45,
 195.5,
 1539.7875,
 1078.15,
 39.1,
 0.0,
 4107.575,
 39.1,
 968.29,
 1559.545,
 2509.125,
 2694.36,
 219.9375,
 1256.48,
 1246.595,
 888.8000000000002,
 68.82,
 39.1,
 917.29,
 1482.75,
 2016.808,
 1635.7,
 1309.85,
 2374.9999999999995,
 2900.575,
 0.0,
 1890.75,
 2779.5550000000007,
 4911.875,
 2004.0,
 1595.17,
 523.905,
 3925.55,
 6233.377500000001,
 4079.1502,
 4312.86,
 2811.3250000000007,
 3802.495,
 3561.7249999999995,
 2195.64,
 1815.6125,
 1596.495,
 977.8,
 1250.5725,
 774.5975,
 415.17,
 0.0,
 1123.345,
 1155.625,
 513.425,
 1164.2675,
 88.30499999999998,
 870.4100000000002,
 840.225,
 379.9425,
 3392.575,
 784.01,
 189.735,
 1848.7,
 951.6114,
 49.425,
 127.365,
 698.73,
 225.705,
 78.86,
 98.3,
 78.75,
 88.745,
 546.955,
 510.17,
 274.745,
 108.285,
 117.52,
 78.52000000000002,
 7

## Teste modelo de Lempel Ziv

In [106]:
#Implementation of the LEmpel Ziv algorithm
def LZ_algo(series):
    Dict_index = {0:series[0]}
    Dict_frequency = {series[0]:1}
    index = 1
    endindex = 1
    while endindex < len(series):
            newstring = series[index:endindex+1]
            if newstring in Dict_index.values():
                endindex += 1
                Dict_frequency[newstring] += 1
            else:
                Dict_index[index] = str(newstring)
                Dict_frequency[newstring] = 1
                endindex += 1
                index = int(endindex)
    return(Dict_index,Dict_frequency)

In [107]:
#Calculation of the Complexity Index for each of the vectors-------------------
LZ = dict()
for key,values in  Vetores_TPV.items():
    s = ''.join(['1' if i > 0 else '0' for i in values])
    LZ[key] = LZ_algo(s)

In [116]:
Populacao_em_analise[0:5]

['23050858000166',
 '7826576806',
 '30844684000179',
 '19702030000178',
 '36742683000108']

In [117]:
LZ['23050858000166']

({0: '1',
  1: '11',
  3: '10',
  5: '111',
  8: '101',
  11: '0',
  12: '1111',
  16: '110',
  19: '1110',
  23: '1101',
  27: '11111',
  32: '01',
  34: '011',
  37: '00',
  39: '0110',
  43: '01100',
  48: '11110',
  53: '0111',
  57: '1100',
  61: '01111',
  66: '001',
  69: '100',
  72: '11001',
  77: '01110',
  82: '111101',
  88: '011111',
  94: '1001',
  98: '1111011',
  105: '111100',
  111: '0011',
  115: '1010'},
 {'1': 20,
  '11': 14,
  '10': 5,
  '111': 8,
  '101': 2,
  '0': 12,
  '1111': 6,
  '110': 4,
  '1110': 1,
  '1101': 1,
  '11111': 1,
  '01': 8,
  '011': 7,
  '00': 3,
  '0110': 2,
  '01100': 1,
  '11110': 4,
  '0111': 4,
  '1100': 2,
  '01111': 2,
  '001': 2,
  '100': 2,
  '11001': 1,
  '01110': 1,
  '111101': 2,
  '011111': 1,
  '1001': 1,
  '1111011': 1,
  '111100': 1,
  '0011': 1,
  '1010': 1})

In [108]:
#Filtrar a populacao com todos os dados necessrios (Provisorio)
Populacao_em_analise = list(Vetores_TPV.keys() - (Vetores_TPV.keys() - Oferta))

In [110]:
#Testes de 'passes' das frequencias--------------------------------------------
PassTests = dict()
PassRates = dict()
for emprestimo in tqdm(Populacao_em_analise):
    saldo = Oferta[emprestimo]['limite']
    juros_mais_iof = (1+Oferta[emprestimo]['taxa_juros'])**(1/30)-1 + 0.000041
    services = 0.02
    retencao = Oferta[emprestimo]['retencao']
    tpv_seletor = sorted([i for i in Vetores_TPV[emprestimo] if i >0][-30:])
    tpv_base = tpv_seletor[int(len(tpv_seletor)/2)] #A mediana dos ultimos trinta valores (ou o tamanho do vetor)
#     print(f'cnpj: {emprestimo} | tpv_base: {tpv_base} | retencao: {retencao} | limite: {saldo} | juros: {juros_mais_iof}')
    pagamento_base = tpv_base * (1-services) * retencao # nao usa?
    PassTests[emprestimo] = dict()
    num = 0
    denom = 0
    for key in LZ[emprestimo][1].keys():
        balanco = float(saldo)
        for action in key:
            balanco = balanco * (1+juros_mais_iof) - int(action) * tpv_base * (1-services) * retencao
        if balanco > saldo:
            PassTests[emprestimo][key] = 'fail'
        else:
            PassTests[emprestimo][key] = 'pass'
            num += len(key) * LZ[emprestimo][1][key]
        denom += len(key) * LZ[emprestimo][1][key]
    PassRates[emprestimo] = num/denom

100%|██████████| 11528/11528 [00:01<00:00, 7046.90it/s]


In [111]:
#Selecao de Candidatos---------------------------------------------------------
Candidatos = [key for key,values in PassRates.items() if values > 0.98 and 17 <= len(LZ[key][1]) <= 32]

Nova_Retencao = dict()
for emprestimo in tqdm(Candidatos):
    saldo = Oferta[emprestimo]['limite']
    juros_mais_iof = (1+Oferta[emprestimo]['taxa_juros'])**(1/30)-1 + 0.000041
    services = 0.02
    retencao = Oferta[emprestimo]['retencao']
    tpv_seletor = sorted([i for i in Vetores_TPV[emprestimo] if i >0][-30:])
    tpv_base = tpv_seletor[int(len(tpv_seletor)/2)] #A mediana dos ultimos trinta valores (ou o tamanho do vetor)
    pagamento_base = tpv_base * (1-services) * retencao
    passrate = PassRates[emprestimo]
    while passrate == PassRates[emprestimo]:
        retencao -= 0.005
        num = 0
        denom = 0
        for key in LZ[emprestimo][1].keys():
            balanco = float(saldo)
            for action in key:
                balanco = balanco * (1+juros_mais_iof) - int(action) * tpv_base * (1-services) * retencao
            if balanco <= saldo:
                num += len(key) * LZ[emprestimo][1][key]
            denom += len(key) * LZ[emprestimo][1][key]
        passrate = num/denom
    Nova_Retencao[emprestimo] = retencao + 0.005
#     print(emprestimo)

100%|██████████| 2115/2115 [00:01<00:00, 1587.75it/s]


In [112]:
len(Nova_Retencao)

2115

In [113]:
df_lz = pd.DataFrame(list(Nova_Retencao.items()), columns=['cnpj_or_cpf','nova_retencao'])

In [114]:
df_lz

,cnpj_or_cpf,nova_retencao
0,36742683000108,0.153434
1,36686823000160,0.153434
2,35447153000166,0.072289
3,28755141000133,0.142519
4,13762920000199,0.108434
...,...,...
2110,22995470000175,0.138434
2111,17392961000191,0.174434
2112,186638345,0.066759
2113,36153663000193,0.059575


In [115]:
output_file_data = data_calculo.replace('-', '')
df_lz.to_csv(f'./outputs/backtest_lz_{tipo_oferta}_{id_oferta}_{output_file_data}.csv', index=False)